### Imports

In [1]:
# import necessary libraries
import pandas as pd
import numpy as np
import os
import glob
from statistics import mean

In [2]:
# Seed value for the project.
seed_value = 42

In [7]:
# Outcome variables - as decided with the host organisation.
# These values correspond to a ventricular fibrillation.
OUTCOMES = [
'ATP in VT/VF delivered',
'ATP One Shot delivered',
'VT1 therapy episodes',
'VF episodes',
'ATP in VT zones started',
'ATP One Shot started',
'ATP in VT zones successful',
'ATP One Shot successful',
'Shocks started',
'Shocks aborted',
'Shocks successful']

# Feature variables specific to patients with single chamber ICDs (ventricular).
VENTR_FEATURES = ['RV pacing impedance [ohm]',
 'RV sensing amplitude (daily mean) [mV]',
 'RV sensing amplitude (daily min.) [mV]',
 'Daily shock lead impedance [ohm]',
 'Right ven. pacing (RVp) [%]',
 'VT1 monitoring episodes',
 'VT1 therapy episodes',
 'VT2 episodes',
 'VF episodes',
 'Episodes during temporary program',
 'ATP in VT zones started',
 'ATP in VT zones successful',
 'ATP One Shot started',
 'ATP One Shot successful',
 'Shocks started',
 'Shocks aborted',
 'Shocks successful',
 'Ineffective ven. max. energy shocks',
 'Mean ventricular heart rate [bpm]',
 'Mean ventricular heart rate at rest [bpm]',
 'Patient activity [% of day]',
 'ATP in VT/VF delivered',
 'ATP One Shot delivered']

### Accessing patient data files

In [8]:
# Accessing the csv files. Each patient is represented by a different file. 
path = os.getcwd() + '/data/the_rest/full_data'
csv_files = glob.glob(os.path.join(path, "*.csv"))


### Defining Cases and Controls

In [9]:
# Generating cases and controls lists of patient IDs. 
case_path = os.getcwd() + '/data/the_rest/the_full_data/Deidentified_data/Cases'
contr_path = os.getcwd() + '/data/the_rest/the_full_data/Deidentified_data/Controls'
case_files = glob.glob(os.path.join(case_path, "*.csv"))
contr_files = glob.glob(os.path.join(contr_path, "*.csv"))

cases = []
controls = []

# Creating a list of cases IDs.
for f in case_files:
    to_replace = case_path + '/'
    case_id = f.replace(to_replace, '')
    case_id = case_id.replace('.csv', '')
    cases.append(int(case_id))

# Creating a list of control IDs.
for f in contr_files:
    to_replace = contr_path + '/'
    contr_id = f.replace(to_replace, '')
    contr_id = contr_id.replace('.csv', '')
    controls.append(int(contr_id))

# Creating a list of all patient IDs.
all_patients = cases + controls

# Checking which patients were not included. 
set(np.array(range(1,226))) - set(all_patients)

{7, 73, 92, 175, 178, 181, 206}

### File integration

In [64]:
# Creating a dataframe to store the integreated files. 
a_data = pd.DataFrame()
time_ser_len = []
  
# Iteraing over the list of csv files
for f in csv_files:
    to_replace = path + '/'
    patient_id = f.replace(to_replace, '')
    patient_id = patient_id.replace('.csv', '')

    # Reading the file
    df = pd.read_csv(f)
    # A list holding all the different lengths available.
    time_ser_len.append(len(df))

    # Adding a patient ID column with the patient's ID. 
    df['patient_id'] = [patient_id for i in range(len(df))]

    # Adding a case/control column
    if int(patient_id) in cases:
        df['case'] = np.full(len(df), 1)
    else:
        df['case'] = np.full(len(df), 0)

    # Concatenating the patients dataframe to the full dataframe. 
    if a_data.empty:
        a_data = df
    else:
        a_data = pd.concat([a_data, df])

### Post-integration cleaning

In [65]:
all_data = a_data.copy()

#### Replacing string values with float values

In [66]:
# Deleting non-numerical symbols:
all_data.loc[:, ("RV sensing amplitude (daily min.) [mV]",
    "RV sensing amplitude (daily mean) [mV]")] = all_data.loc[:, ("RV sensing amplitude (daily min.) [mV]",
    "RV sensing amplitude (daily mean) [mV]")].astype('str').replace('> 20.0', 20)

all_data.loc[:, ("RV sensing amplitude (daily min.) [mV]",
    "RV sensing amplitude (daily mean) [mV]")] = all_data.loc[:, ("RV sensing amplitude (daily min.) [mV]",
    "RV sensing amplitude (daily mean) [mV]")].astype('str').replace('< 2.0', 2)

all_data.loc[:, ("RA sensing amplitude (daily min.) [mV]",
"RA sensing amplitude (daily mean) [mV]")] = all_data.loc[:, ("RA sensing amplitude (daily min.) [mV]",
"RA sensing amplitude (daily mean) [mV]")].astype('str').replace('> 8.0', 8)

all_data.loc[:, ("RA sensing amplitude (daily min.) [mV]",
"RA sensing amplitude (daily mean) [mV]")] = all_data.loc[:, ("RA sensing amplitude (daily min.) [mV]",
"RA sensing amplitude (daily mean) [mV]")].astype('str').replace('< 0.5', 0.5)

# YES replaced with 1
all_data.loc[:, ("Atrial arrhythmia ongoing at end of mon. interv.",
"Long ongoing atrial episode detected", 'ATP One Shot delivered')] = all_data.loc[:, 
("Atrial arrhythmia ongoing at end of mon. interv.",
"Long ongoing atrial episode detected", 'ATP One Shot delivered')].astype('str').replace('YES', 1)

# NO replaced with 0
all_data.loc[:, ("Atrial arrhythmia ongoing at end of mon. interv.",
"Long ongoing atrial episode detected", 'ATP One Shot delivered')] = all_data.loc[:, 
("Atrial arrhythmia ongoing at end of mon. interv.",
"Long ongoing atrial episode detected", 'ATP One Shot delivered')].astype('str').replace('NO', 0)

# Replacing >110 to 110
all_data.loc[:, 'Mean PVC/h'] = all_data.loc[:, 
'Mean PVC/h'].astype('str').replace('> 110', 110)

# Replace >65534 with 65534
all_data.loc[:, 'Atrial monitoring episodes'] = all_data.loc[:, 
'Atrial monitoring episodes'].astype('str').replace('> 65534', 65534)

# Replace OFF with NaN:
all_data.loc[:, ("Thoracic impedance daily mean [ohm]", "RA pacing impedance [ohm]")] = all_data.loc[:, 
("Thoracic impedance daily mean [ohm]", "RA pacing impedance [ohm]")].astype('str').replace('OFF', np.nan)

# Replace OFF with NaN:
all_data.loc[:, ('Atrial burden [% of day]')] = all_data.loc[:, 
('Atrial burden [% of day]')].astype('str').replace('OFF', np.nan)

# Replace "OFF" with NaN:
all_data.loc[:, ('Long ongoing atrial episode detected')] = all_data.loc[:, 
('Long ongoing atrial episode detected')].astype('str').replace('OFF', np.nan)

# Values than contain "AUTO":
repl_auto = all_data['RA pulse amplitude [V]'][all_data['RA pulse amplitude [V]'].str.contains("AUTO", na=False)].to_list()
for i in repl_auto:
    all_data.loc[:, ('RA pulse amplitude [V]')] = all_data.loc[:, 
    ('RA pulse amplitude [V]')].astype('str').replace(i, float(i[:-5]))
    
# NaN to 0
all_data.loc[:, ('ATP in VT/VF delivered', 'ATP One Shot delivered')] = all_data.loc[:, 
('ATP in VT/VF delivered', 'ATP One Shot delivered')].astype('str').replace('nan', 0)

# Deleting non-numerical symbols:
all_data.loc[:, ('RV pacing impedance [ohm]')] = all_data.loc[:, 
('RV pacing impedance [ohm]')].astype('str').replace('< 200', 200)



In [67]:
# Deleting non-numerical symbols:
all_data.loc[:, ('RA pacing impedance [ohm]')] = all_data.loc[:, 
('RA pacing impedance [ohm]')].astype('str').replace('> 3000', 3000)

# Replacing OFF with NaN.
all_data.loc[:, ('VT1 zone limit [ms]', 'VT2 zone limit [ms]')] = all_data.loc[:, 
('VT1 zone limit [ms]', 'VT2 zone limit [ms]')].astype('str').replace('OFF', np.nan)

#### Converting columns to float or integer datatype

In [68]:
# Conberted to integers:
all_data['patient_id'] = all_data['patient_id'].astype(int)
all_data['Shocks successful'] = all_data['Shocks successful'].astype(int)

# Converted to float values:
all_data['RA sensing amplitude (daily mean) [mV]'] = all_data['RA sensing amplitude (daily mean) [mV]'].astype(float)
all_data['RA sensing amplitude (daily min.) [mV]'] = all_data['RA sensing amplitude (daily min.) [mV]'].astype(float)
all_data['RV sensing amplitude (daily mean) [mV]'] = all_data['RV sensing amplitude (daily mean) [mV]'].astype(float)
all_data['RV sensing amplitude (daily min.) [mV]'] = all_data['RV sensing amplitude (daily min.) [mV]'].astype(float)
all_data['Atrial burden [% of day]'] = all_data['Atrial burden [% of day]'].astype(float)
all_data['Atrial monitoring episodes'] = all_data['Atrial monitoring episodes'].astype(float)
all_data['Mean PVC/h'] = all_data['Mean PVC/h'].astype(float)
all_data['Atrial arrhythmia ongoing at end of mon. interv.'] = all_data['Atrial arrhythmia ongoing at end of mon. interv.'].astype(float)
all_data['Long ongoing atrial episode detected'] = all_data['Long ongoing atrial episode detected'].astype(float)
all_data['Thoracic impedance daily mean [ohm]'] = all_data['Thoracic impedance daily mean [ohm]'].astype(float)
all_data['ATP One Shot delivered'] = all_data['ATP One Shot delivered'].astype(float)
all_data['ATP in VT/VF delivered'] = all_data['ATP in VT/VF delivered'].astype(float)
all_data['RV pacing impedance [ohm]'] = all_data['RV pacing impedance [ohm]'].astype(float)
all_data['RA pulse amplitude [V]'] = all_data['RA pulse amplitude [V]'].astype(float)
all_data['RA pacing impedance [ohm]'] = all_data['RA pacing impedance [ohm]'].astype(float)
all_data['VT1 zone limit [ms]'] = all_data['VT1 zone limit [ms]'].astype(float)
all_data['VT2 zone limit [ms]'] = all_data['VT2 zone limit [ms]'].astype(float)
all_data['VF zone limit [ms]'] = all_data['VF zone limit [ms]'].astype(float)

#### Comparing ICD settings between cases and controls

In [69]:
grouped = all_data.groupby('patient_id')

# Storing the settings in lists:
cas_vt1 = []
cas_vt2 = []
cas_vf = []
con_vt1 = []
con_vt2 = []
con_vf = []

# Iterating through patient IDs and their data.
for pt_id, pt_dt in grouped:
    # Dropping NaN values in the parameter columns
    cols_of_interest = pt_dt[['VT1 zone limit [ms]', 'VT2 zone limit [ms]', 'VF zone limit [ms]']].dropna(axis=0)
    # Taking a mean of each column.
    (vt1, vt2, vf) = cols_of_interest.mean(axis=0)

    # If case, mean added to appropriate list.
    if pt_id in cases:
        cas_vt1.append(vt1)
        cas_vt2.append(vt2)
        cas_vf.append(vf)

    # If control, mean added to appropriate list.
    elif pt_id in controls:
        con_vt1.append(vt1)
        con_vt2.append(vt2)
        con_vf.append(vf)

# Creating a list of values corresponding to the 3 different parameters.
param_case_contr = []
for i in [cas_vt1, cas_vt2, cas_vf, con_vt1, con_vt2, con_vf]:
    param_case_contr.append(np.array(i)[~np.isnan(np.array(i))].mean())

### Excluding parameters with missing data (specific to atrial leads) 
Columns specific to atrial leads will be dropped.

In [70]:
# Creating a dataframe with percentages of missing values per column.
percent_missing = all_data.isnull().sum() * 100 / len(all_data)
missing_value_df = pd.DataFrame({'column_name': all_data.columns,
                                 'percent_missing': percent_missing})
missing_value_df.sort_values('percent_missing', inplace=True)

# Dropping columns that have over 30% of missing data - atria specific columns.
cols_to_drop = missing_value_df[missing_value_df['percent_missing']>30]['column_name'].to_list()
all_data.drop(cols_to_drop, axis=1, inplace=True)

### Excluding patients with >30% missing data per key variable.

In [71]:
# Checking which of the outcome variables have missing data.
# Patients will be excluded, if there is missing data in key variables.
miss_days = all_data.reset_index().copy()
miss_days[VENTR_FEATURES].isna().sum()

RV pacing impedance [ohm]                       0
RV sensing amplitude (daily mean) [mV]       1628
RV sensing amplitude (daily min.) [mV]       1628
Daily shock lead impedance [ohm]                2
Right ven. pacing (RVp) [%]                     0
VT1 monitoring episodes                         0
VT1 therapy episodes                            0
VT2 episodes                                    0
VF episodes                                     0
Episodes during temporary program               0
ATP in VT zones started                         0
ATP in VT zones successful                      0
ATP One Shot started                            0
ATP One Shot successful                         0
Shocks started                                  0
Shocks aborted                                  0
Shocks successful                               0
Ineffective ven. max. energy shocks             0
Mean ventricular heart rate [bpm]               0
Mean ventricular heart rate at rest [bpm]     105


In [72]:
# Grouping patients by patient ID.
grouped = miss_days.groupby('patient_id')

# Initiating lists to hold patient IDs.
thor_miss = []
rv_miss = []
mean_miss = []

# Iterating through patients, recording if they have >30% missing data. 
for i, pt_df in grouped:
    # Right ventricle impedance.
    if pt_df['RV sensing amplitude (daily mean) [mV]'].isna().sum()/len(pt_df) >0.3:
        rv_miss.append(i)
        print("RV", i)
    # Significant variable in previous studies
    elif pt_df['Mean ventricular heart rate at rest [bpm]'].isna().sum()/len(pt_df) >0.3:
        mean_miss.append(i)
        print("Mean", i)

# Joining the lists.
pt_miss_data = thor_miss + rv_miss + mean_miss
len(set(pt_miss_data))

# Excluding these patients from the dataframe. 
for i in pt_miss_data:
    miss_days = miss_days[miss_days.patient_id != i]

RV 86
RV 93
RV 96
RV 155
RV 224


### Identifying consecutive missing days in the data

In [73]:
# Sorting the data based on patient ID and transmission day.
miss_days.sort_values(by=['patient_id', 'Transmission received on (days)'], inplace=True)

# Grouping the data by patient ID.
grouped = miss_days.groupby('patient_id')

# Calculating the differences between consecutive transmission days values.
miss_days['time_gap'] = grouped['Transmission received on (days)'].diff().abs()

# Identifying the biggest gaps in each patient's data.
biggest_gaps = miss_days.groupby('patient_id')['time_gap'].max()

# Getting the index values for two largest time gaps.
nearest_values = miss_days.groupby('patient_id').apply(lambda group: group.nlargest(2, 'time_gap')).reset_index(drop=True)

# Counting the total amount of missed days per patient.
missed_days_per_patient = miss_days.groupby('patient_id')['time_gap'].sum()

In [74]:
# Sorting to deal with the biggest gaps through editing the original files. 
gaps = pd.DataFrame(biggest_gaps)
gaps.sort_values('time_gap', ascending=False)[:30]

,time_gap
patient_id,
30,14.998658
40,14.124491
65,14.124398
75,14.040637
56,14.040475
199,13.999398
125,13.998588
64,13.998403
195,13.997674


In [75]:
nearest_values[['Transmission received on (days)', 'patient_id', 'time_gap']][nearest_values['time_gap']>13].sort_values('time_gap', ascending=False)

,Transmission received on (days),patient_id,time_gap
56,-2565.055266,30,14.998658
76,-2561.871470,40,14.124491
126,-2272.028854,65,14.124398
146,-2034.056262,75,14.040637
108,-2390.981262,56,14.040475
378,-500.083843,199,13.999398
238,-231.997095,125,13.998588
124,-2136.074769,64,13.998403
370,-807.951852,195,13.997674
230,-1288.911690,121,13.997338


### Dropping transmissions received on the same day

In [76]:
# Converting the transmission reseived on days values to an integer index. 
miss_days['Transmission received on (days)'] = miss_days['Transmission received on (days)'].abs().round(1)
miss_days['Transmission received on (days)'] = miss_days['Transmission received on (days)'].apply(np.floor).astype(int)
miss_days.sort_values(by=['patient_id', 'Transmission received on (days)'], inplace=True)

# Droping rows if the data was already received on the same day - keeping first.
miss_days = miss_days.drop_duplicates(subset=['patient_id', 'Transmission received on (days)'], keep='first')

### Imputing data

In [77]:
# Group the data by patient_id
grouped = miss_days.groupby('patient_id')

# Creating a new DataFrame with the complete sequence of days for each patient
complete_data = pd.DataFrame()
for patient_id, group in grouped:
    # Getting the full range of days
    min_day = group['Transmission received on (days)'].min()
    max_day = group['Transmission received on (days)'].max()+1
    # Creating a dataframe of the full range
    all_days = np.arange(min_day, max_day)
    idx = np.arange(0, len(all_days))
    patient_ar = np.full(len(all_days), patient_id)
    patient_data = pd.DataFrame({'Transmission received on (days)': all_days, 'patient_id': patient_ar, 'index': idx})
    # Adding patient data to the full data
    complete_data = pd.concat([complete_data, patient_data])

# Merging the full range dataframe to the original, to include missed days.
imputed_data = complete_data.merge(miss_days, on=['Transmission received on (days)', 'patient_id'], how='left')

# Last observation carried forward imputation.
imputed_data = imputed_data.groupby('patient_id').apply(lambda x : x.ffill()).reset_index(drop=True)

# Cleaning the index values
imputed_data.drop('index_y', axis=1, inplace=True)
imputed_data.rename(columns={'index_x':'index'}, inplace=True)

In [78]:
# Counting the mean length and standard deviation of patient data.
lengths = [len(imputed_data[imputed_data['patient_id']==i]) for i in imputed_data.patient_id.unique()]
print(mean(lengths))
np.std(lengths)

355.716894977169


123.0545123196666

### Outcome: ATP, ICD shock events summarised in the 'shock' column
Converting the outcome variables (n=11) to 0 or 1 in 'shock' column.

In [79]:
# Creating an outcome dataframe with all the features.
outcome_df = imputed_data[['patient_id', 'case', 'index']+VENTR_FEATURES].copy().reset_index()

In [80]:
# Grouping patients by their ID.
patient_groups = outcome_df.groupby('patient_id')

# Creating a dataframe to count the number of outcomes in the data.
outcome_count = pd.DataFrame()

# Looping through patient dataframes.
for _, patient_df in patient_groups:
    # Calculating the difference between consecutive rows, fill NaN with 0.
    diffs = patient_df[OUTCOMES].diff(axis=0).fillna(0)

    # If any increase in score between days, adding 1 in 'shock' column.
    has_shock = diffs.gt(0).any(axis=1).astype(int)
    
    diffs = diffs.gt(0)
    diffs['patient_id'] = patient_df['patient_id']
    outcome_count = pd.concat([outcome_count, diffs])

    # Updating the 'shock' column.
    outcome_df.loc[patient_df.index, 'shock'] = has_shock

#### Creating a table with the number of outcome values

In [81]:
#  Number of patients affected by each outcome
num_affect = [len(outcome_count[outcome_count[i]==1].patient_id.unique()) for i in OUTCOMES]
np.array(num_affect)

# Dataframe of events and patients affected.
out_count = pd.DataFrame(index=outcome_count.sum()[:-1].index)
out_count['Total Events'] = outcome_count.sum()[:-1]
out_count['Patients Affected'] = num_affect
out_count

,Total Events,Patients Affected
ATP in VT/VF delivered,3,3
ATP One Shot delivered,10,8
VT1 therapy episodes,32,5
VF episodes,35,27
ATP in VT zones started,150,22
ATP One Shot started,30,20
ATP in VT zones successful,140,20
ATP One Shot successful,10,9
Shocks started,47,29
Shocks aborted,14,11


In [82]:
# Checking the total number of events
outcome_df['shock'].sum()

182.0

#### Checking for incorrect labelling of cases and controls.

In [83]:
# Cases that had shocks:
true_cas = outcome_df['patient_id'].loc[outcome_df["case"] == 1].loc[outcome_df['shock']==1].unique()

# All recorded cases:
all_cas = outcome_df['patient_id'].loc[outcome_df["case"] == 1].unique()

# Difference between ture and recorded cases. Highlights cases without shocks.
set(all_cas) - set(true_cas)

{176}

In [58]:
# Controls that had shocks:
outcome_df['patient_id'].loc[outcome_df["case"] == 0].loc[outcome_df['shock']==1].unique()

array([], dtype=int64)

In [75]:
# Looking for the specific day of when the control had the shock. 
outcome_df[['index', 'patient_id', 'case', 'shock']].loc[outcome_df['patient_id']==169].loc[outcome_df["case"] == 1].loc[outcome_df['shock']==1]

,index,patient_id,case,shock
58637,267,169,1.0,1.0


In [76]:
# Saving the cleaned dataframe:
outcome_df.to_csv('cleaned_df.csv')